<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/StackExchange_GoogleNews_WordEmbedding_MultiLabelClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  
  basepath = '/content/drive/My Drive/NLP'
  sys.path.append('/content/drive/My Drive/NLP/custom-functions')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade gensim -qq


In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/NLP/custom-functions']

In [ ]:
#Importing the required libraries
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, multilabel_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
#importing classes from custom packages
import custom_preprocessor_mod as cp
from plot_learning_curve import plot_learning_curve as plt


In [ ]:
import gensim
from gensim.models import KeyedVectors
import joblib

In [ ]:
#Checking the genism version
gensim.__version__

'4.3.1'

In [ ]:
#Defined the google drive folders for accessing/saving models related files
embeddings_folder = Path(basepath)/ 'assignment7/WordEmbeddings'
data_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'
model_saving_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'

Gensim Vectorizer

In [ ]:
from gensim_vectorizer  import GensimVectorizer  #Imported the GensimVectorizer

In [ ]:
import gensim.downloader as loader
#w2vec_google_news = loader.load("word2vec-google-news-300")
#!unzip '/content/drive/MyDrive/data/WordEmbeddings/wiki-news-300d-1M-subword.vec.zip' -d data_folder

In [ ]:
data_cleaned = data_folder/ "df_multilabel_hw_cleaned.joblib"
stackExchange_dataset = joblib.load(data_cleaned)

In [ ]:
type(stackExchange_dataset)

pandas.core.frame.DataFrame

In [ ]:
stackExchange_dataset.shape

(47427, 3)

In [ ]:
stackExchange_dataset.columns

Index(['cleaned_text', 'Tags', 'Tag_Number'], dtype='object')

In [ ]:
stackExchange_dataset.head(5)

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]"
3,run python script php server run nginx web ser...,php python,"[2, 7]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]"


In [ ]:
X = stackExchange_dataset['cleaned_text'].values
y = stackExchange_dataset['Tag_Number'].values
print(type(X), type(y))
print(X.dtype, y.dtype)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
object object


In [ ]:
!pip install swifter -qq

In [ ]:
import swifter
import ast
stackExchange_dataset['Tag_Number_list'] = stackExchange_dataset['Tag_Number'].swifter.apply(lambda x: ast.literal_eval(x))

Pandas Apply:   0%|          | 0/47427 [00:00<?, ?it/s]

In [ ]:
stackExchange_dataset['Tag_Number_list'][0]

[0, 9]

In [ ]:
y_final = stackExchange_dataset['Tag_Number_list'].values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer as mlb
y_stackExchange_encoding = mlb().fit_transform(y_final)

In [ ]:
type(y_stackExchange_encoding)

numpy.ndarray

In [ ]:
y_stackExchange_encoding.shape

(47427, 10)

In [ ]:
from sklearn.model_selection import train_test_split
X_sExchange_train, X_valid_test, y_sExchange_train, y_valid_test = train_test_split(X, y_stackExchange_encoding, test_size=0.4, random_state=42)
X_sExchange_valid, X_sExchange_test, y_sExchange_valid, y_sExchange_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=42)

In [ ]:
#Checking the length of X (train, test & valid)
print(len(X_sExchange_train), len(X_sExchange_test), len(X_sExchange_valid))

28456 9486 9485


In [ ]:
#Checking the shape of y (train, test & valid)
print(y_sExchange_train.shape, y_sExchange_test.shape, y_sExchange_valid.shape)

(28456, 10) (9486, 10) (9485, 10)


In [ ]:
pretrained_sExchange_file = str(embeddings_folder/ "model_stackExchange_CBOW.bin")
sExchange_vectors = KeyedVectors.load(pretrained_sExchange_file)


In [ ]:
type(sExchange_vectors)

gensim.models.keyedvectors.KeyedVectors

In [ ]:
!pip install OneVsRestClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement OneVsRestClassifier (from versions: none)
ERROR: No matching distribution found for OneVsRestClassifier


In [ ]:
#pipeline using sklearn
from sklearn.multiclass import OneVsRestClassifier 
se_pipeline=Pipeline(
    [
      ("vectorizer", GensimVectorizer(sExchange_vectors)),
      ("clf", OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1))
    ]
)

Hyperparameter Tuning

In [ ]:
#se_param_grid = {
  #  "regressor__C": [10000000000],
#}

#Defining the GridSearch with Cross Validation
#se_gridsearchcv = GridSearchCV(
  #  se_pipeline, se_param_grid, cv=5, return_train_score=True, n_jobs=-1)

In [ ]:
se_pipeline.fit(X_sExchange_train, y_sExchange_train)

Pipeline(steps=[('vectorizer',
                 GensimVectorizer(pretrained_vectors=<gensim.models.keyedvectors.KeyedVectors object at 0x7fe87469ce80>)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(solver='sag'),
                                     n_jobs=1))])

In [ ]:
#Checking the training score
print("Training score: ", se_pipeline.score(X_sExchange_train, y_sExchange_train))

Training score:  0.6438009558616812


In [ ]:
#Checking the test score
print("Test score: ", se_pipeline.score(X_sExchange_test, y_sExchange_test))

Test score:  0.629348513598988


In [ ]:
y_test_pred = se_pipeline.predict(X_sExchange_test)

In [ ]:
print("Hamming loss: ", hamming_loss(y_sExchange_test, y_test_pred))

Hamming loss:  0.06720430107526881


In [ ]:
print("Classification report: \n", classification_report(y_sExchange_test, y_test_pred))

Classification report: 
               precision    recall  f1-score   support

           0       0.84      0.62      0.72      2123
           1       0.89      0.77      0.82      1686
           2       0.85      0.32      0.47      1152
           3       0.87      0.93      0.90      5417
           4       0.92      0.80      0.85      1528
           5       0.88      0.91      0.90      4934
           6       0.90      0.15      0.26       169
           7       0.91      0.08      0.14       130
           8       1.00      0.01      0.03        72
           9       0.89      0.69      0.78      2502

   micro avg       0.88      0.79      0.83     19713
   macro avg       0.89      0.53      0.59     19713
weighted avg       0.88      0.79      0.81     19713
 samples avg       0.83      0.80      0.80     19713



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.svm import LinearSVC
se_svc_pipeline = Pipeline([
    ('vectorizer', GensimVectorizer(sExchange_vectors)),
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))
])

In [ ]:
se_svc_pipeline.fit(X_sExchange_train, y_sExchange_train)

Pipeline(steps=[('vectorizer',
                 GensimVectorizer(pretrained_vectors=<gensim.models.keyedvectors.KeyedVectors object at 0x7fe87469ce80>)),
                ('clf', OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1))])

In [ ]:
#Checking training score
#print(se_svc_pipeline.score(X_sExchange_train, y_sExchange_train))

In [ ]:
#print(se_svc_pipeline.score(X_sExchange_test, y_sExchange_test))

In [ ]:
svc_test_pred = se_svc_pipeline.predict(X_sExchange_test)

In [ ]:
print("Accuracy Score: ", accuracy_score(y_sExchange_test, svc_test_pred))

Accuracy Score:  0.6611849040691545


In [ ]:
print("Hamming loss: ", hamming_loss(y_sExchange_test, svc_test_pred))

Hamming loss:  0.059413873076112164


In [ ]:
multilabel_confusion_matrix(svc_test_pred, y_sExchange_test)
#print("f1 score for test: ", f1_test)

array([[[7070,  643],
        [ 293, 1480]],

       [[7651,  301],
        [ 149, 1385]],

       [[8226,  641],
        [ 108,  511]],

       [[3343,  383],
        [ 726, 5034]],

       [[7850,  210],
        [ 108, 1318]],

       [[4001,  399],
        [ 551, 4535]],

       [[9316,   95],
        [   1,   74]],

       [[9355,   85],
        [   1,   45]],

       [[9414,   66],
        [   0,    6]],

       [[6725,  617],
        [ 259, 1885]]])

In [ ]:
print("Classification report: \n", classification_report(y_sExchange_test, svc_test_pred))

Classification report: 
               precision    recall  f1-score   support

           0       0.83      0.70      0.76      2123
           1       0.90      0.82      0.86      1686
           2       0.83      0.44      0.58      1152
           3       0.87      0.93      0.90      5417
           4       0.92      0.86      0.89      1528
           5       0.89      0.92      0.91      4934
           6       0.99      0.44      0.61       169
           7       0.98      0.35      0.51       130
           8       1.00      0.08      0.15        72
           9       0.88      0.75      0.81      2502

   micro avg       0.88      0.83      0.85     19713
   macro avg       0.91      0.63      0.70     19713
weighted avg       0.88      0.83      0.84     19713
 samples avg       0.86      0.83      0.84     19713



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
